In [ ]:
# default_exp db110

# db110

> Classes and functions for the db110

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export
import pandas as pd
import numpy as np
import regex
#import ipdb
from pandas import DataFrame
import matplotlib.pyplot as plt




## Functions

In [ ]:
#export
def getBIT_Status(line):
    '''
    Input:
        line = line containing a BIT status change
    Output:
        A two element tuple containing:
            1. BIT Test represented as Component.TestName
            2. BIT Status
    '''
    componentName = getString(line,'AS node name = ','  TID = ')
    testName = getString(line,'  TID = ',', BIT status = ')
    testStatus = getString(line,'BIT status = ','\' BitMgr.c')
    return componentName+'.'+testName,testStatus

In [ ]:
def genBIT_Changes(fn):
    '''
    Input:
        fn = Name of Error Log
    Output:
        List where each element has 4 items:
            1. Epoch time since Jan 1, 1970 (sec)
            2. Event time adjusted by Time Sync (TSY) to be elapsed time (sec)
            3. Name of BIT test of form: Component.TestName
            4. Updated Status      
    '''
    bs_list = []    #BIT Status Change List
    fh = open(fn,'r')
    syncValue = None
    #debug_here()
    for line in fh:
        if line.find('TR_Rare TSY|') != -1:
            debug_here()
            if syncValue == None:
                #debug_here()
                syncValue = getTimeSync(line)
                print (syncValue)
            else:
                newSyncValue = getTimeSync(line)
                if abs(newSyncValue)>1000000:
                    syncValue = newSyncValue
                else:
                    syncValue += newSyncValue
                    print ('new '+syncValue)

        elif line.find('BIT status = ') != -1:
            #debug_here()
            if syncValue != None:
                bitTest,bitStatus = getBIT_Status(line)
                rawTime = string.atoi(line.split()[0])
                evtTime = rawTime+syncValue
                #debug_here()
                if bitTest == 'SSR.CommError':
                    #debug_here()
                    #print bitTest    
                    #debug_here()
                    bs_list.append([rawTime,evtTime,bitTest,bitStatus])
        else:
            pass
        
    fh.close()
    return bs_list


In [ ]:
#export
def getFieldNames(logfile):
    "Returns field names from a db110 datalog collected by GTS"
    nameList = []
    fields = []
    pattern = r'#\s\d*\s(?P<field>\w.*)' 
    fieldsRegex = regex.compile(pattern)
    #fieldsRegex = regex.compile(pattern)
    fh = open(logfile,'r')
    lines = fh.readlines()
    for line in lines:
        m = fieldsRegex.search(line)
        if m:
            field = m.group('field')
            field = field.replace(' ','_')
            field = field.replace(':','.')
            field = field.replace('.','_')
            nameList.append(field)
        else:
            pass
    fh.close()
    #debug_here()
    # try a list here in prep for pandas
    
    # this looks like a set up for my matlab routines
    #fieldNames = list(enumerate(nameList))
    
    # how about this 
    fieldNames = nameList
        
    # need a dictionary?
    #fieldNames = dict.fromkeys(nameList)
    
    return fieldNames

In [ ]:
names = getFieldNames('PitchT3.log')

In [ ]:
print(names)

['row_number', 'Absolute_sample_number', 'pitchGimbal_run_curMode', 'pitchGimbal_run_curPosition', 'pitchGimbal_run_setPosition', 'pitchGimbal_run_errPosition', 'pitchGimbal_run_curVelocity', 'pitchGimbal_run_velError', 'pitchGimbal_run_setVelocity', 'pitchGimbal_run_errVelocity', 'pitchGimbal_run_curCommand']


Graphing with fastbook module; Use to describe a test process or sequence

## Classes

## Test